## Financial Statements Coding Tool

By Ken Burchfiel

Released under the MIT license

In [1]:
import pandas as pd
import numpy as np
import os
import time
start_time = time.time()
import matplotlib.pyplot as plt
statements_top_folder = 'simulated_spending_data'
data_output_folder = 'coded_transactions'
expense_codes_path = 'sample_finance_codes.csv'

In [2]:
expense_codes = pd.read_csv(expense_codes_path).drop('Notes', axis = 1)

expense_codes

,Code,Subcode,Code_Description,Subcode_Description
0,A,A-X,ATM,ATM Withdrawals
1,B,B-C,Basics,Convenience Stores
2,B,B-B,Basics,Books
3,B,B-F,Basics,Pets
4,B,B-G,Basics,Gifts
...,...,...,...,...
99,W,W-S,Clothing,Shoes
100,W,W-X,Clothing,Clothing--Other
101,X,X-X,Other,Other Expenses
102,Z,Z-C,Excluded,Credit Card Payments


In [3]:
statements_file_list = []
for root, dirs, files in os.walk(statements_top_folder): statements_file_list.extend([{"name":name, "path":os.path.join(root, name)} for name in files])
# See https://docs.python.org/3/library/os.html#os.walk
df_statements = pd.DataFrame(statements_file_list)

df_statements['Account_Type'] = df_statements['name'].str.split('-').str[0]
df_statements['file_type'] = df_statements['name'].str.split('_').str[0]
df_statements.reset_index(drop=True,inplace=True)
df_statements

,name,path,Account_Type,file_type
0,amex_1-simulated_2022_data.csv,simulated_spending_data\amex_1-simulated_2022_...,amex_1,amex
1,truist_1-simulated_2022_data.csv,simulated_spending_data\truist_1-simulated_202...,truist_1,truist


In [4]:
def process_amex_statement(file_path, account_type = None):
    expected_column_list = ['Date', 'Description', 'Amount']
    df_csv = pd.read_csv(file_path)
    if df_csv.columns.to_list() != expected_column_list:
        return ValueError("Columns in .csv file are different than expected")
    new_column_order = ['Date', 'Amount', 'Description']
    df_csv = df_csv[new_column_order]
    df_csv['Date'] = pd.to_datetime(df_csv['Date'])
    df_csv['Account_Type'] = account_type

    return df_csv

df_amex_example = process_amex_statement(df_statements.iloc[0,1], account_type = df_statements.iloc[0,2])
df_amex_example

,Date,Amount,Description,Account_Type
0,2022-05-06,32.94,310 BOWERY 0000 NEW YORK NY,amex_1
1,2022-11-21,0.98,6331177 - DUNKIN DONFLUSHING NY,amex_1
2,2022-01-09,132.30,AIRBNB SAN FRANCISCO CA,amex_1
3,2022-11-06,3.20,ALHEGAZY HALAL FOOD NEW YORK NY,amex_1
4,2022-11-08,11.03,Arts and Crafts BeerNew York NY,amex_1
...,...,...,...,...
249,2022-09-28,86.06,UNIVERSITY HARDWARE NEW YORK NY,amex_1
250,2022-07-20,21.95,VIZCAYA MUSEUM AND GMIAMI FL,amex_1
251,2022-04-21,11.87,WESTSIDE MARKET,amex_1
252,2022-02-14,2.92,WESTSIDE MARKET 2840NEW YORK NY,amex_1


In [5]:
def process_truist_statement(file_path, account_type = None):
    expected_column_list = ['Date',
 'Transaction Type',
 'Check/Serial #',
 'Description',
 'Amount',
 'Daily Posted Balance']
    # expected_column_list = ['Date', 'Description', 'Amount']
    df_csv = pd.read_csv(file_path)
    if df_csv.columns.to_list() != expected_column_list:
        return ValueError("Columns in .csv file are different than expected")
    # if df_csv.columns.to_list() != expected_column_list:
    #     return ValueError("Columns in .csv file are different than expected")
    # new_column_order = ['Date', 'Amount', 'Description']
    # df_csv = df_csv[new_column_order]
    new_column_list = ['Date', 'Amount', 'Description']
    df_csv = df_csv[new_column_list]
    # The Truist values represent negative numbers with parentheses. The 
    # following lines of code replace the parentheses with a negative
    # sign and also remove the dollar sign.
    df_csv['Amount'] = df_csv['Amount'].astype('str')
    df_csv['Amount'] = df_csv['Amount'].str.replace(')','', regex = False) # Removes parentheses
    # and dollar signs from values so that they can be converted to numbers
    df_csv['Amount'] = df_csv['Amount'].str.replace('$','', regex = False)
    df_csv['Amount'] = df_csv['Amount'].str.replace('(','-', regex = False)
    df_csv['Amount'] = df_csv['Amount'].str.replace('(','-', regex = False) 
    df_csv['Amount'] = df_csv['Amount'].astype(float)
    df_csv['Amount'] = df_csv['Amount']*-1 # Truist makes expenses 
    # negative by default, but I'm changing them to positive numbers
    df_csv['Date'] = pd.to_datetime(df_csv['Date'])
    df_csv['Account_Type'] = account_type

    return df_csv

i = 2
df_truist_example = process_truist_statement(df_statements.iloc[1,1], account_type = df_statements.iloc[1,2])
df_truist_example

,Date,Amount,Description,Account_Type
0,2022-09-29,-0.52,NEW YORK NY JOES GOURMET DELI FOREIGN ATM SURC...,truist_1
1,2022-07-11,62.23,Roman Catholic C INTERNET PAYMENT,truist_1
2,2022-05-30,386.15,ACH PMT AMEX EPAYMENT INTERNET PAYMENT,truist_1
3,2022-07-10,70.52,ACH PMT AMEX EPAYMENT INTERNET PAYMENT,truist_1
4,2022-07-19,139.45,ACH PMT AMEX EPAYMENT INTERNET PAYMENT,truist_1
...,...,...,...,...
60,2022-12-04,66.79,SOCIETY LITTLE FLO DEBIT CARD PURCHASE,truist_1
61,2022-01-22,73.04,SOCIETY LITTLE FLO DEBIT CARD PURCHASE,truist_1
62,2022-10-23,28.48,ST PATRICK CHURCH DEBIT CARD PURCHASE,truist_1
63,2022-07-03,2.40,TAXI CORDOBA CORDOBA DEBIT CARD PURCHASE,truist_1


In [6]:
def create_expenses_df(df_statements):
    df_list = []
    for i in range(len(df_statements)):
        file_type = df_statements.loc[i, 'file_type']
        path = df_statements.loc[i, 'path']
        account_type = df_statements.loc[i, 'Account_Type']
        if file_type == 'amex':
            df_list.append(process_amex_statement(file_path = path, account_type = account_type))
        elif file_type == 'truist':
            df_list.append(process_truist_statement(file_path = path, account_type = account_type))
        else:
            print("Function not in place for account type. Skipping DataFrame")
    return df_list

In [7]:
df_expenses_list = create_expenses_df(df_statements=df_statements)

In [8]:
df_finances = pd.concat([df for df in df_expenses_list])
# df_finances.drop_duplicates(inplace=True) # Note: this was dropped because
# sometimes duplicate expenses are valid (e.g. you might use the subway
# twice in one day, or you might have two identical flight records if you
# book two plane tickets.)
df_finances['Subcode'] = ''
df_finances['Month'] = df_finances['Date'].dt.month
df_finances

,Date,Amount,Description,Account_Type,Subcode,Month
0,2022-05-06,32.94,310 BOWERY 0000 NEW YORK NY,amex_1,,5
1,2022-11-21,0.98,6331177 - DUNKIN DONFLUSHING NY,amex_1,,11
2,2022-01-09,132.30,AIRBNB SAN FRANCISCO CA,amex_1,,1
3,2022-11-06,3.20,ALHEGAZY HALAL FOOD NEW YORK NY,amex_1,,11
4,2022-11-08,11.03,Arts and Crafts BeerNew York NY,amex_1,,11
...,...,...,...,...,...,...
60,2022-12-04,66.79,SOCIETY LITTLE FLO DEBIT CARD PURCHASE,truist_1,,12
61,2022-01-22,73.04,SOCIETY LITTLE FLO DEBIT CARD PURCHASE,truist_1,,1
62,2022-10-23,28.48,ST PATRICK CHURCH DEBIT CARD PURCHASE,truist_1,,10
63,2022-07-03,2.40,TAXI CORDOBA CORDOBA DEBIT CARD PURCHASE,truist_1,,7


In [9]:
pd.set_option('display.max_rows', len(df_finances))
purchases_by_description = df_finances.groupby('Description')['Amount'].count().reset_index()
purchases_by_description.sort_values('Amount', ascending = False, inplace = True)
purchases_by_description.head(5)

,Description,Amount
83,NYCT PAYGO NEW YORK NY,105
2,ACH PMT AMEX EPAYMENT INTERNET PAYMENT,19
85,ONLINE PAYMENT - THANK YOU,11
87,PAYMENT VENMO INTERNET PAYMENT,10
32,DELTA AIR LINES,6


In [10]:
pd.set_option('display.max_rows', 50)

In [11]:
expenses_by_description = df_finances.groupby('Description')['Amount'].sum().reset_index()
expenses_by_description.sort_values('Amount', ascending = False, inplace = True)
expenses_by_description

,Description,Amount
2,ACH PMT AMEX EPAYMENT INTERNET PAYMENT,5387.10
115,TREAS DRCT TREASURY DIRECT AAAA ACH DEBIT,1050.00
32,DELTA AIR LINES,551.79
19,CHECK #1337,550.68
95,Rent PAYMENT ID ZELLE PAYMENT TO,549.55
...,...,...
127,Work Paystub,-332.00
14,CASHOUT VENMO ACH CREDIT,-687.02
77,MONEYLINE FID BKG SVC LLC ACH CREDIT,-855.00
76,MOBILE PAYMENT - THANK YOU,-931.63


In [12]:
def add_subcodes(df, subcode, expense_descriptors):
    initial_count = df.query("Subcode != ''")['Subcode'].count()
    df['Subcode'] = np.where(df['Description'].str.lower().str.contains(expense_descriptors.lower(), regex = True) == True, subcode, df['Subcode'])
    new_count = df.query("Subcode != ''")['Subcode'].count()
    print(f"Added {subcode} to {new_count - initial_count} entries.") 
    return df

# For the use of '|' to check multiple examples, see:
# https://pandas.pydata.org/docs/reference/api/pandas.Series.str.contains.html


# An alternative to np.where is series.where. Regarding Series.where syntax, 
# see: https://pandas.pydata.org/docs/reference/api/pandas.Series.where.html
# As described in the documentation, this syntax is somewhat the opposite of np.where. In cases where the condition is true, the column will be kept the same; in cases where it differs, the column will be changed.

The following list allows you to specify springs that correspond to different categories. Be careful to backslash out any regex symbols present in different descriptions. For a list of these symbols, see: https://docs.python.org/3/library/re.html

In [13]:
code_description_pairs_list = []

# ATM withdrawals:
code_description_pairs_list.append(("A-X", " atm "))

# Convenience stores:
code_description_pairs_list.append(("B-C", "duane reade|cvs|rite aid"))

# Books:
code_description_pairs_list.append(("B-B", "libreria san pablo|abebooks|magnificat"))

# Pets:
code_description_pairs_list.append(("B-F", "petco"))

# Gifts:
code_description_pairs_list.append(("B-G", "flowers by valli"))

# Taxis and ridesharing:
code_description_pairs_list.append(("B-R", "uber|lyft|curb taxi app|gett|taxi cordoba|nyc taxi|taxi tmc"))

# Shipping: 
code_description_pairs_list.append(("B-S", "usps"))

# Subway and other public transit (in area where you live):
code_description_pairs_list.append(("B-T", "nyct paygo|omnypyg|mnr etix ticket|nyc ferry"))

# Other basics:
code_description_pairs_list.append(("B-X", "klean kanteen"))


# Dining out:
code_description_pairs_list.append(("D-X", "pizza|koronet|dostoros|dos toros|king of falafel|jin 00|columbia university new york|arts and crafts beer|mel's burger bar|diginn|sanfords|junzi kitchen|nyc bagel & coffee|madisoncafe|madison cafe|haagen-dazs|milano market|pita hot|taste of italy|angry orchard cidery|astoria taco factory|baum 00034|blue bottle coffee|butterfunnew|kennedy fried chicke|palace restaurant|flafel on broadway|junko sushi|birch coffee|by the way bakery|cafe metro|citi field concessio|connollys pub|daruma|first on first deli|five guys ny|food hall d|frankie's dogs on the|friedmans|garnet wines and liq|gglpay columbia univ|sip a cup|hamilton deli|hooda halal|hula poke|katsuhama|lili and loo|La Salle Dumpling|mottley kitchen|mottsu|arepas grill|athens grill|peking garden|shake shack         new york|samad's|senza gluten|sweetgreen columbia|toms restaurant|famousfamiglia|symposium restaurantnew york|shake shack - 1197|ample hills|cho dang gol 30new york|dear mama- westn|go go curry 300n|le pain quotidinew york|la masa|alhegazy halal food|310 bowery|osaka japanese|the bronx breweb|the heights ny new york|the thirsty koaastoria|baum 30052"))

# Investment withdrawals:
code_description_pairs_list.append(("D-X", "orale.*tacos"))

# Sporting events:
code_description_pairs_list.append(("E-S", "stubhub"))

# Newspapers:
code_description_pairs_list.append(("E-W", "wall-st-journal"))

# Alcohol:
code_description_pairs_list.append(("F-A", "international wines|angela's wine"))

# Grocery stores:
code_description_pairs_list.append(("F-G", "westside mark|morton willia|trade fair|city fresh market|h mart|tokyo market"))

# Vending machines:
code_description_pairs_list.append(("F-V", "cmsvend|city limits vending"))

# Home storage:
code_description_pairs_list.append(("H-C", "the container store"))

# House furnishings:
code_description_pairs_list.append(("H-F", "university hardware|home depot|homedepot|ikea|lowes"))

# Moving:
code_description_pairs_list.append(("H-M", "u-haul"))

# Rent:
code_description_pairs_list.append(("H-R", "rent payment"))

# Investment withdrawals:
code_description_pairs_list.append(("J-S", "moneyline.*credit"))

# YouTube earnings:
code_description_pairs_list.append(("J-Y", "adsense|youtube_pa"))

# Phone payments:
code_description_pairs_list.append(("K-P", "t-mobile"))

# Crowdsourced donations:
code_description_pairs_list.append(("L-G", "gofundme"))

# Charitable donations:
code_description_pairs_list.append(("L-X", "catholic charities|Consumer myeoffering|catholic relief se|focus|roman cathol    new york|Roman Catholic C|educando by worldf|society little flo"))

# Eye care:
code_description_pairs_list.append(("M-E", "lenscrafters|sp lenses for less|gammarayoptix"))

# Video/Computer Games:
code_description_pairs_list.append(("P-G", "nintendo|pmdg simulations|steampowered|typeracer"))

#Air travel:
code_description_pairs_list.append(("T-A", "delta|united airlines|southwest airlines|american airlines|lufthansa|frontier|jetblue"))

# Bus Travel:
code_description_pairs_list.append(("T-B", "njt pabt|wanderu|washington deluxe bub"))

# Travel-related entertainment:
code_description_pairs_list.append(("T-E", "frost science|vizcaya museum|sky views miami"))

# Food during trips:
code_description_pairs_list.append(("T-F", "fish tales|peak food court|1914 by kolben|farmer's fridge|paschals concourse batlanta|cafe castro|grapevine micro-mark|cafe on the ave|comida buena|farmer brown|frontera grill|herberer term|hooked on the vine|hotel alhambra|duke's lake union ch|hotel on booking|k-1 cafeteria|kroger mid atl|lgad bisoux|moxies grill and barmiami|whataburger|smashburger b7c ewr|tomasita's|izanami 0004|northside socia|atl 6065 low country"))

# Hotels:
code_description_pairs_list.append(("T-H", "airbnb"))

# Parking on trips:
code_description_pairs_list.append(("T-P", "dfw airport parking"))

# Public transit as part of trips:
code_description_pairs_list.append(("T-S", "njt rail|renfe virtual internmadrid|metro de madrid|metrocard/airtrain|smartrip|sound transit|kfir train|clipper systems|dc transit service cmiami"))

# Clothing:
code_description_pairs_list.append(("W-C", "robbie & co"))

# Clothing:
code_description_pairs_list.append(("W-S", "clarks"))


# Items to exclude from budget analysis (e.g. so as not to incur double counting:)
code_description_pairs_list.append(("Z-C", "payment - thank you|ach pmt amex epayment|payment to credit card"))

code_description_pairs_list.append(("Z-X", "mobile deposit"))


# code_description_pairs_list.append(("", ""))

# code_description_pairs_list.append(("", ""))



In [14]:
for pair in code_description_pairs_list:
    df_finances = add_subcodes(df = df_finances, subcode = pair[0], expense_descriptors = pair[1])

Added A-X to 3 entries.
Added B-C to 7 entries.
Added B-B to 2 entries.
Added B-F to 0 entries.
Added B-G to 4 entries.
Added B-R to 5 entries.
Added B-S to 0 entries.
Added B-T to 115 entries.
Added B-X to 1 entries.
Added D-X to 38 entries.
Added D-X to 0 entries.
Added E-S to 1 entries.
Added E-W to 3 entries.
Added F-A to 2 entries.
Added F-G to 9 entries.
Added F-V to 8 entries.
Added H-C to 0 entries.
Added H-F to 8 entries.
Added H-M to 1 entries.
Added H-R to 3 entries.
Added J-S to 3 entries.
Added J-Y to 0 entries.
Added K-P to 0 entries.
Added L-G to 1 entries.
Added L-X to 15 entries.
Added M-E to 1 entries.
Added P-G to 2 entries.
Added T-A to 9 entries.
Added T-B to 0 entries.
Added T-E to 1 entries.
Added T-F to 4 entries.
Added T-H to 1 entries.
Added T-P to 0 entries.
Added T-S to 4 entries.
Added W-C to 1 entries.
Added W-S to 0 entries.
Added Z-C to 32 entries.
Added Z-X to 1 entries.


In [15]:
df_finances.query("Subcode == ''")['Description'].value_counts().head(40)

PAYMENT VENMO INTERNET PAYMENT                            10
CASHOUT VENMO ACH CREDIT                                   2
6331177 - DUNKIN DONFLUSHING            NY                 1
BEST BUY            NEW YORK            NY                 1
ST PATRICK CHURCH DEBIT CARD PURCHASE                      1
RIO METRO REGIONAL ALBUQUERQUE NM  DEBIT CARD PURCHASE     1
Receivable Work ACH CREDIT                                 1
Work Paystub                                               1
LOCKANDENJOY.COM 06-14 SEVILLA  DEBIT CARD PURCHASE        1
CHECK #1337                                                1
CABILDO 06-12 GRANADA  DEBITCARD PURCHASE                  1
CABILDO 06-12 GRANADA  DEBITCARD INT'L TRAN FEE            1
TST* FRANKLIN PARK                                         1
T2 BOOK STORE       QUEENS              NY                 1
RUBYS CANDY & GROCER                                       1
MENE INC            NEW YORK            NY                 1
MANUEL J REGALOS 208CORD

In [16]:
df_finances

,Date,Amount,Description,Account_Type,Subcode,Month
0,2022-05-06,32.94,310 BOWERY 0000 NEW YORK NY,amex_1,D-X,5
1,2022-11-21,0.98,6331177 - DUNKIN DONFLUSHING NY,amex_1,,11
2,2022-01-09,132.30,AIRBNB SAN FRANCISCO CA,amex_1,T-H,1
3,2022-11-06,3.20,ALHEGAZY HALAL FOOD NEW YORK NY,amex_1,D-X,11
4,2022-11-08,11.03,Arts and Crafts BeerNew York NY,amex_1,D-X,11
...,...,...,...,...,...,...
60,2022-12-04,66.79,SOCIETY LITTLE FLO DEBIT CARD PURCHASE,truist_1,L-X,12
61,2022-01-22,73.04,SOCIETY LITTLE FLO DEBIT CARD PURCHASE,truist_1,L-X,1
62,2022-10-23,28.48,ST PATRICK CHURCH DEBIT CARD PURCHASE,truist_1,,10
63,2022-07-03,2.40,TAXI CORDOBA CORDOBA DEBIT CARD PURCHASE,truist_1,B-R,7


In [17]:
# Number of expenses coded so far (and percentage of all expenses that have been coded):
print("Expenses Coded:",df_finances.query("Subcode != ''")['Subcode'].count())
print("{:.2%}".format(df_finances.query("Subcode != ''")['Subcode'].count()/len(df_finances)))
print("Uncoded Expenses:", df_finances.query("Subcode == ''")['Subcode'].count())

Expenses Coded: 285
89.34%
Uncoded Expenses: 34


In [18]:
df_finances.sort_values(['Subcode', 'Description'], inplace = True)
df_finances

,Date,Amount,Description,Account_Type,Subcode,Month
1,2022-11-21,0.98,6331177 - DUNKIN DONFLUSHING NY,amex_1,,11
5,2022-11-23,47.54,BEST BUY NEW YORK NY,amex_1,,11
21,2022-03-13,0.35,CABILDO 06-12 GRANADA DEBITCARD INT'L TRAN FEE,truist_1,,3
22,2022-10-30,7.03,CABILDO 06-12 GRANADA DEBITCARD PURCHASE,truist_1,,10
23,2022-12-26,-73.35,CASHOUT VENMO ACH CREDIT,truist_1,,12
...,...,...,...,...,...,...
213,2022-05-04,-17.34,ONLINE PAYMENT - THANK YOU,amex_1,Z-C,5
214,2022-07-03,-123.91,ONLINE PAYMENT - THANK YOU,amex_1,Z-C,7
215,2022-10-29,-163.57,ONLINE PAYMENT - THANK YOU,amex_1,Z-C,10
216,2022-09-21,-78.51,ONLINE PAYMENT - THANK YOU,amex_1,Z-C,9


The following cell creates a list of all duplicated entries (with the first entries) removed. You can check over this list to help ensure that you didn't have any incorrect duplicates in your original data. Note that duplicate expenses are often valid.

If you do find an incorrect duplicate, you can remove it when creating the edited version of your finances .csv file.

In [19]:
duplicate_entries = df_finances[df_finances.duplicated()].copy()
duplicate_entries.sort_values(['Subcode', 'Date', 'Amount', 'Description'], inplace = True)
duplicate_entries.to_csv(data_output_folder+'\\duplicated_rows.csv', index = False)

In [20]:
df_finances.to_csv(data_output_folder+'\\finances_updated_in_python.csv', index = False)

In [21]:
end_time = time.time()

run_time = end_time - start_time
run_minutes = run_time // 60
run_seconds = run_time % 60
print("Completed run at",time.ctime(end_time),"(local time)")
print("Total run time:",'{:.3f}'.format(run_time),"second(s) ("+str(run_minutes),"minute(s) and",'{:.3f}'.format(run_seconds),"second(s))") # Only valid when the program is run nonstop from start to finish

Completed run at Tue Nov  1 23:31:09 2022 (local time)
Total run time: 1.826 second(s) (0.0 minute(s) and 1.826 second(s))
